In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

### Ranking general

In [ ]:
# Abrir parquet en un DataFrame
df_ranking = pd.read_parquet("../../data/raw/fbref_ranking_general.parquet")

# Codificacion equipo
data = {'equipo': ["Real Madrid", "Barcelona", "Atlético", "Sevilla", "Betis",
    "Villarreal", "Real Sociedad", "Valencia", "Osasuna", "Getafe",
    "Rayo Vallecano", "Celta", "Alavés", "Las Palmas", "Mallorca",
    "Granada", "Cádiz", "Girona", "Leganés", "Espanyol"]}
df = pd.DataFrame(data)

print("Antes del encoding:")
print(df)

# Crear el codificador
encoder = OneHotEncoder(sparse_output=False)

# Ajustar y transformar
encoded = encoder.fit_transform(df[['equipo']])

# Crear nuevo DataFrame con nombres de columnas
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['equipo']))

# Concatenar con el original (opcional)
df_final = pd.concat([df, encoded_df], axis=1)

print("\nDespués del One-Hot Encoding:")
print(df_final)


# Eliminar columnas innecesarias
df_ranking = df_ranking.drop(columns=["Attendance", "Notes"])

print(df_ranking.head())

   Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG  xGA  xGD  \
0   1        Barcelona   7  6  1  0  21   5  16   19    2.71  15.8  6.3  9.5   
1   2      Real Madrid   7  6  0  1  16   8   8   18    2.57  14.2  8.6  5.6   
2   3       Villarreal   7  5  1  1  13   5   8   16    2.29  12.3  7.1  5.2   
3   4            Elche   7  3  4  0  10   6   4   13    1.86   9.1  7.4  1.7   
4   5  Atlético Madrid   7  3  3  1  14   9   5   12    1.71  12.9  6.7  6.2   

   xGD/90     Last 5                        Top Team Scorer  \
0    1.35  D W W W W  Robert Lewandowski, Ferrán Torres - 4   
1    0.81  W W W W L                      Kylian Mbappé - 8   
2    0.74  D L W W W                     Tajon Buchanan - 3   
3    0.24  W D W D W              André Silva, Rafa Mir - 3   
4    0.89  D W D W W                     Julián Álvarez - 6   

               Goalkeeper  
0             Joan García  
1        Thibaut Courtois  
2  Luiz Lúcio Reis Júnior  
3           Matías Dituro  


### Ranking home/away

In [4]:
df_ranking_ha = pd.read_parquet("../../data/raw/fbref_ranking_home_away.parquet")

for col in df_ranking_ha.columns:
    col_clean = col.strip().replace("(", "").replace(")", "").replace("'", "")
    parts = [p.strip() for p in col_clean.split(",")]
    
    if len(parts) == 2:
        if parts[0] == "Home":
            new_col = parts[1] + "_home"
        elif parts[0] == "Away":
            new_col = parts[1] + "_away"
        else:
            new_col = parts[1]
    else:
        new_col = parts[0]
    
    df_ranking_ha = df_ranking_ha.rename(columns={col: new_col})

print(df_ranking_ha.head())

   Rk            Squad  MP_home  W_home  D_home  L_home  GF_home  GA_home  \
0   1        Barcelona        3       3       0       0       11        1   
1   2      Real Madrid        3       3       0       0        5        1   
2   3       Villarreal        4       4       0       0       10        1   
3   4            Elche        4       3       1       0        6        2   
4   5  Atlético Madrid        4       3       1       0       11        5   

   GD_home  Pts_home  ...  L_away  GF_away  GA_away  GD_away  Pts_away  \
0       10         9  ...       0       10        4        6        10   
1        4         9  ...       1       11        7        4         9   
2        9        12  ...       1        3        4       -1         4   
3        4        10  ...       0        4        4        0         3   
4        6        10  ...       1        3        4       -1         2   

   Pts/MP_away  xG_away  xGA_away  xGD_away  xGD/90_away  
0         2.50      7.8       4.9

### Players stats

In [5]:
# Abrir parquet en un DataFrame
df_players = pd.read_parquet("../../data/raw/fbref_players_stats.parquet")

# Eliminar columnas innecesarias
df_players = df_players.drop(columns=["Rk", "Nation", "Matches"])

print(df_players.head())

                Player    Pos       Squad     Age  Born MP Starts  Min  90s  \
0  Jones El-Abdellaoui  MF,DF  Celta Vigo  19-256  2006  1      0   34  0.4   
1          Abdel Abqar     DF      Getafe  26-199  1999  3      3  252  2.8   
2           Akor Adams     FW     Sevilla  25-239  2000  4      3  286  3.2   
3    David Affengruber     DF       Elche  24-190  2001  5      5  450  5.0   
4   Julen Agirrezabala     GK    Valencia  24-273  2000  6      6  540  6.0   

  Gls  ... Gls-90min Ast-90min G+A-90min G-PK-90min G+A-PK-90min xG-90min  \
0   0  ...      0.00      0.00      0.00       0.00         0.00     0.00   
1   0  ...      0.00      0.00      0.00       0.00         0.00     0.02   
2   0  ...      0.00      0.31      0.31       0.00         0.31     0.31   
3   0  ...      0.00      0.00      0.00       0.00         0.00     0.00   
4   0  ...      0.00      0.00      0.00       0.00         0.00     0.00   

  xAG-90min xG+xA-90min npxG-90min npxG+xAG-90min  
0      0.1

### Results

In [ ]:
# Abrir parquet en un DataFrame
df_matches = pd.read_parquet("../../data/raw/fbref_results.parquet")

# Eliminar columnas innecesarias
df_matches = df_matches.drop(columns=["Date", "Match Report", "Notes"])
print(df_matches.head())

    Wk  Day   Time        Home   xG Score  xG.1            Away  Attendance  \
0  1.0  Fri  19:00      Girona  0.6   1–3   3.4  Rayo Vallecano     12403.0   
1  1.0  Fri  21:30  Villarreal  2.4   2–0   0.9          Oviedo     18333.0   
2  1.0  Sat  19:30    Mallorca  0.2   0–3   2.1       Barcelona     23318.0   
3  1.0  Sat  21:30      Alavés  1.2   2–1   0.9         Levante     12837.0   
4  1.0  Sat  21:30    Valencia  2.0   1–1   0.8   Real Sociedad     45333.0   

                           Venue             Referee  
0  Estadi Municipal de Montilivi     Javier Alberola  
1         Estadio de la Cerámica     Alejandro Muñíz  
2       Estadi Mallorca Son Moix   José Luis Munuera  
3        Estadio de Mendizorroza        Miguel Sesma  
4            Estadio de Mestalla  Jose Maria Sánchez  


### Squads stats

In [ ]:
# Abrir parquet en un DataFrame
df_squad = pd.read_parquet("../../data/raw/fbref_squads_stats.parquet")

# Eliminar columnas innecesarias
# df_ranking = df_ranking.drop(columns=["Date", "Match Report", "Notes"])
print(df_squad.head())

  ('Unnamed: 0_level_0', 'Squad')  ('Unnamed: 1_level_0', '# Pl')  \
0                          Alavés                              22   
1                   Athletic Club                              23   
2                 Atlético Madrid                              21   
3                       Barcelona                              22   
4                           Betis                              25   

   ('Unnamed: 2_level_0', 'Age')  ('Unnamed: 3_level_0', 'Poss')  \
0                           27.8                            49.5   
1                           27.7                            52.7   
2                           27.8                            59.3   
3                           24.9                            69.6   
4                           27.9                            50.2   

   ('Playing Time', 'MP')  ('Playing Time', 'Starts')  \
0                       6                          66   
1                       6                          66   
2    

### Player-matches

In [ ]:
# Abrir parquet en un DataFrame
df_matches_players = pd.read_parquet("../../data/raw/fbref_player_matches.parquet")

# Eliminar columnas innecesarias
# df_ranking = df_ranking.drop(columns=["Date", "Match Report", "Notes"])
print(df_matches_players.head())

         Date  Day     Comp        Round Venue Result       Squad    Opponent  \
0  2025-08-17  Sun  La Liga  Matchweek 1  Home  L 0–2  Celta Vigo      Getafe   
1  2025-08-23  Sat  La Liga  Matchweek 2  Away  D 1–1  Celta Vigo    Mallorca   
2  2025-08-27  Wed  La Liga  Matchweek 6  Home  D 1–1  Celta Vigo       Betis   
3  2025-08-31  Sun  La Liga  Matchweek 3  Home  D 1–1  Celta Vigo  Villarreal   
4  2025-09-14  Sun  La Liga  Matchweek 4  Home  D 1–1  Celta Vigo      Girona   

  Start                                  Pos  ...  \
0     N                                RM,RB  ...   
1     N  On matchday squad, but did not play  ...   
2     N  On matchday squad, but did not play  ...   
3     N  On matchday squad, but did not play  ...   
4     N  On matchday squad, but did not play  ...   

                            Passes_Att                          Passes_Cmp%  \
0                                   26                                 80.8   
1  On matchday squad, but did not pl